# ACI Create New User

This example shows how to create a new controller user.

We need to follow a workflow to gather the info needed for the POST request.
Additionally, each input needs to be validated before posting, else we need to capture server responses and handle them correctly. 

Things we should do before requesting input:
 - Get all users from APIC.
 - Get all roles from APIC.
 - Get all security domains from APIC.

When we take input, we can then present a list of roles and domains to select from, rather than allowing free-form typing.

In [1]:
# Set up all the modules we need.
import requests
import json
import pandas as pd
import os
import getpass
import sys

In [7]:
def get_info():
    # Read controller host/IP and auth credentials from a config file, if available. 
    # Test for config.py and import the variables.
    # Check if the variable is imported; if not, prompt for input.
    if os.path.isfile('./config.py'):
        from config import controller, username, password

    try:
        controller
    except NameError:
        controller = input("Controller Hostname/IP: ")

    try:
        username
    except NameError:
        username = input("APIC Username: ")
    
    try: 
        password 
    except NameError:
        password = getpass.getpass("APIC Password: ")

    return controller, username, password

In [8]:
controller, username, password = get_info()

In [4]:
print("Controller: " + controller)
print("Username: " + username)
print("Password: " + password)

Controller: 10.18.188.101
Username: api-andrew
Password: b1n2m3v4c5x6z7^^


In [27]:
def auth(controller, username, password):
    # Shhh! We don't want self-signed cert warnings.
    requests.packages.urllib3.disable_warnings() 
    # Set up the request and pass the authentication data as JSON
    s = requests.session()
    response = s.post(auth_url, json=auth_data, verify=False)
    return response.status_code
    #s.status_code

In [23]:
# Set up our URL
base_url = "https://" + str(controller) + "/api/"
auth_class = "aaaLogin.json"
auth_url = base_url + auth_class
    # Setup auth data as a dictionary 
auth_data = {
    "aaaUser":{
        "attributes":{
            "name":username,
            "pwd":password
        }
    }
}
print(auth_url)
print(auth_data)

https://10.18.188.101/api/aaaLogin.json
{'aaaUser': {'attributes': {'name': 'api-andrew', 'pwd': 'b1n2m3v4c5x6z7^^'}}}


In [31]:
reponse = auth(controller, username, password)
print(response)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Response'

In [30]:
if response != 200:
    print("Didn't authenticate.")
    print("HTTP Status: " + str(response))


Didn't authenticate.
HTTP Status: <Response [200]>


In [86]:
# Set up our user class URLs.
# We set up one for GETting data (inlcuding child objects)
# and one for POSTing data.

qstring = "?rsp-subtree=children"
# Construct the user post URL
user_class="node/mo/uni/userext.json"
post_user_url = base_url + user_class
get_user_url = get_user_url + qstring
print(get_user_url)
print(post_user_url)

https://10.18.188.101/api/node/mo/uni/userext.json?rsp-subtree=children?rsp-subtree=children?rsp-subtree=children?rsp-subtree=children
https://10.18.188.101/api/node/mo/uni/userext.json


In [87]:
def get_users():
    users = s.get(get_user_url, verify=False)
    users = users.json()
    print(json.dumps(users, indent=4, sort_keys=True))
    dn = users['imdata'][0]['aaaUserEp']['attributes']['dn']
    name_list = []
    firstname_list = []
    lastname_list = []
    status_list = []
    phone_list = []
    users_list = users['imdata'][0]['aaaUserEp']['children']
    for user in users_list:
        if "aaaUser" in user: 
            #print(user)
            rn = user['aaaUser']['attributes']['rn']
            name = user['aaaUser']['attributes']['name']  
            firstname = user['aaaUser']['attributes']['firstName']
            lastname = user['aaaUser']['attributes']['lastName']
            status = user['aaaUser']['attributes']['accountStatus']
            phone = user['aaaUser']['attributes']['phone']
            name_list.append(name)
            firstname_list.append(firstname)
            lastname_list.append(lastname)
            phone_list.append(phone)
            status_list.append(status)
    zlist = zip(status_list, name_list, firstname_list, lastname_list, phone_list, rn_list)
    df_input = list(zlist)
    df = pd.DataFrame(df_input, columns=("Status","User Name","First Name","Last Name","Phone","RN"))

In [88]:
def get_roles():
    roles = [1, 2, 3]
    return roles

In [89]:
def get_domains():
    domains = [1,2,3]
    return domains

In [90]:
def get_new_user():
    os.system('cls' if os.name == 'nt' else 'clear')
    prompt = input("Add New User? (y/N)")
    if prompt.lower() in ('n', 'N'):
        sys.exit()
    elif prompt.lower() in ('y', 'Y'):
        get_user_input()
    else:
        get_new_user()

In [92]:
def get_user_input():
    print("There's no error checking here.") 
    print("Just stop/re-run the program if you mistype anything.")
    user = input("Username: ")
    print("Password needs three of the following: ")
    print(" - lower case letter")
    print(" - upper case letter")
    print(" - number")
    print(" - symbol")
    print(" ")
    password1 = getpass.getpass("Password: ")
    password2 = getpass.getpass("Re-enter Password: ")
    if password1 != password2:
        print("Passwords are not the same. DO OVER!")
        sys.exit()

    firstname = input("[Optional] First Name: ")
    lastname = input("[Optional] Last Name: ")
    phone = input("[Optional] Phone: ")
    email = input("[Optional] Email: ")
    print("Enter security domains, with a [Y]es or [N]o: ")
    domain_all = input("Security Domain - All?  (y/n): ")
  
    if domain_all.lower() in ('y','Y','Yes','YES'):
        domain_all_priv = input("[R]ead or [W]rite: ")

    domain_common = input("Security Domain - Common? (y/n): ")
    if domain_common.lower() in ('y','Y','Yes','YES'):
        domain_common_priv = input("[R]ead or [W]rite: ")

    domain_mgmt = input("Security Domain - Mgmt? (y/n): ")
    if domain_mgmt.lower() in ('y','Y','Yes','YES'):
        domain_mgmt_priv = input("[R]ead or [W]rite: ")

    expires = input("Expires? [y/n]")
    activate = input("Activate User? [y/n]")
    

In [93]:
get_roles()

[1, 2, 3]

In [95]:
get_domains()

[1, 2, 3]

In [96]:
get_users()

{
    "imdata": [
        {
            "error": {
                "attributes": {
                    "code": "8",
                    "text": "rsp-subtree argument is incorrect (possible options: _no_|children|full)"
                }
            }
        }
    ],
    "totalCount": "1"
}


KeyError: 'aaaUserEp'

In [65]:
# Prompt for user info
prompt = input("Add New User? (y/N)")

Add New User? (y/N)n


In [40]:
if prompt.lower() not in ('y', 'Y'):
    sys.exit()

In [43]:
print("There's no error checking here.") 
print("Just stop/re-run the program if you mistype anything.")
user = input("Username: ")
print("Password needs three of the following: ")
print(" - lower case letter")
print(" - upper case letter")
print(" - number")
print(" - symbol")
print(" ")
password1 = getpass.getpass("Password: ")
password2 = getpass.getpass("Re-enter Password: ")
if password1 != password2:
    print("Passwords are not the same. DO OVER!")
    sys.exit()
firstname = input("[Optional] First Name: ")
lastname = input("[Optional] Last Name: ")
phone = input("[Optional] Phone: ")
email = input("[Optional] Email: ")
print("Enter security domains, with a [Y]es or [N]o: ")
domain_all = input("Security Domain - All?  (y/n): ")
if domain_all.lower() in ('y','Y','Yes','YES'):
    domain_all_priv = input("[R]ead or [W]rite: ")

domain_common = input("Security Domain - Common? (y/n): ")
if domain_common.lower() in ('y','Y','Yes','YES'):
    domain_common_priv = input("[R]ead or [W]rite: ")

domain_mgmt = input("Security Domain - Mgmt? (y/n): ")
if domain_mgmt.lower() in ('y','Y','Yes','YES'):
    domain_mgmt_priv = input("[R]ead or [W]rite: ")

expires = input("Expires? [y/n]")
activate = input("Activate User? [y/n]")




There's no error checking here.
Just stop/re-run the program if you mistype anything.
Username: schmuck
Password: ········
Re-enter Password: ········
[Optional] First Name: Jody is
[Optional] Last Name: a schmuck
[Optional] Email: jody-da-schmuck@mailinator.com
Enter security domains, with a [Y]es or [N]o: 
Security Domain - All?  (y/n): y
[R]ead or [W]rite: w
Security Domain - Common? (y/n): y
[R]ead or [W]rite: w
Security Domain - Mgmt? (y/n): y
[R]ead or [W]rite: w
Expires? [y/n]n
Activate User? [y/n]y
Username: schmuck
Password: ***** 
First Name: schmuck
Last Name: a schmuck
Email: jody-da-schmuck@mailinator.com
Security Domain - All: y
Security Domain - All: w
Security Domain - Common: y
Security Domain - Common: w
Security Domain - Mgmt: y
Security Domain - Mgmt: w
Expires: n
Active: y


In [66]:
print("Username: " + str(user))
print("Password: ***** ")
print("First Name: " + str(firstname))
print("Last Name: " + str(lastname))
print("Email: " + str(phone))
print("Email: " + str(email))
print("Security Domain - All: " + str(domain_all))
print("Security Domain - All: " + str(domain_all_priv))
print("Security Domain - Common: " + str(domain_common))
print("Security Domain - Common: " + str(domain_common_priv))
print("Security Domain - Mgmt: " + str(domain_mgmt))
print("Security Domain - Mgmt: " + str(domain_mgmt_priv))
print("Expires: " + str(expires))
print("Active: " + str(activate))

Username: admin
Password: ***** 
First Name: admin
Last Name: 
Email: jody-da-schmuck@mailinator.com
Security Domain - All: y
Security Domain - All: w
Security Domain - Common: y
Security Domain - Common: w
Security Domain - Mgmt: y
Security Domain - Mgmt: w
Expires: n
Active: y


In [ ]:
userext = "uni/userext.json"

## User POST Data
```
{
            "aaaUser": {
                "attributes": {
                    "name": user,
                    "firstName": firstname,
                    "lastName": lastname,
                    "phone": phone,
                    "email": email,
                },
                "children": [{
                    "aaaUserCert": {
                        "attributes": {
                            "name": "userabc.crt",
                            "data": "-----BEGIN CERTIFICATE-----\nMIICjjCCAfegAwIBAgIJAMQnbE <snipped content> ==\n-----END CERTIFICATE-----",
                        },
                        "children": []
                    },
                    "aaaUserDomain": {
                        "attributes": {
                            "name": "all",
                        },
                        "children": [{
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "aaa",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "access-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "fabric-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "nw-svc-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "ops",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "read-all",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "tenant-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "tenant-ext-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }, {
                            "aaaUserRole": {
                                "attributes": {
                                    "name": "vmm-admin",
                                    "privType": "writePriv",
                                },
                                "children": []
                            }
                        }]
                    }
                }]
            }
        }
```